### Billbords

A try to extract data from WIkibase.cloud [SPARQL](https://sweopendata.wikibase.cloud/wiki/Kommuner)

* This Notebook  
  * [SPARQL query](https://tinyurl.com/2dwdoq8q)


In [3]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://sweopendata.wikibase.cloud/query/sparql" 
# Query https://tinyurl.com/2dwdoq8q
queryAnslagstavla = """
PREFIX wb: <https://sweopendata.wikibase.cloud/entity/>

SELECT  ?r ?s  WHERE {?r ?s wb:Q240} """


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WikibaseAnslagstavla = get_sparql_dataframe(endpoint_url, queryAnslagstavla)
WikibaseAnslagstavla.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   r       0 non-null      object
 1   s       0 non-null      object
dtypes: object(2)
memory usage: 0.0+ bytes


In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
WikibaseAnslagstavla.to_csv("Anslagstavlor.csv")